In [1]:
from qiskit import QuantumCircuit, Aer, execute, IBMQ
from qiskit.providers import JobStatus
from qiskit.providers.ibmq import least_busy
from qiskit.tools.monitor import job_monitor
from qiskit.providers.aer.noise import NoiseModel
from qiskit.tools.visualization import plot_histogram
from math import atan2, sqrt, pi
import import_ipynb
#import probabilities as prob

importing Jupyter notebook from probabilities.ipynb


In [19]:
#statesStayHome, statesTests, statesCases = prob.get_lesser_model_states()
#basic_graph = prob.get_conditional_probability(statesStayHome, statesTests, statesCases)[2]

"""
Order of Cases conditional probabilities in graph: 
P(C=0|A=0,B=0), P(C=1|A=0,B=0), P(C=0|A=0,B=1), P(C=1|A=0,B=1), P(C=0|A=1,B=0), P(C=1|A=1,B=0), P(C=0|A=1,B=1), P(C=1|A=1,B=1)
From Fig 4 in https://arxiv.org/pdf/2004.14803.pdf -- 
our order is first col first row, first col second row, second col first row, etc. 
so the first two probabilities add up to one, the third and fourth add up to one, etc. 
"""
#print(basic_graph)
oil_graph = {'IR': ([], [.75, .25]), # From 2004.14803, Fig 10
         'SM': (['IR'], [.3, .7, .8, .2]), #P(0|!A), P(1|!A), P(0|A), P(1|A)
         'OI': ([], [.6, .4]),
         'SP': (['OI', 'SM'], [.9, .1, .5, .5, .4, .6, .2, .8])
        }


In [20]:
#TODO assertAlmostEqual(angle_from_probability(0.2, 0.8), 2.2143)
def angle_from_probability(p0, p1):
    # from 2004.14803 equation 20
    angle = 2 * atan2(sqrt(p1), sqrt(p0))
    return angle

In [6]:
import math

def find_num_states(numprobs, in_edges):
    if in_edges == 0: # if parent node
        numstates = numprobs
    elif in_edges > 0: # if child node
        numstates = (numprobs/(2**(in_edges+1))) * 2
    return numstates

def find_numstates_parents(graph, parentvarnames):
    """Eqn 21 from https://arxiv.org/pdf/2004.14803.pdf"""
    numstates_parents = 0
    for i in range(len(parentvarnames)):
        parent = parentvarnames[i]
        parentprobs = graph[parent]
        numprobs = len(parentprobs[1]) # number of states if parent node, number of conditional states if child node
        in_edges = len(graph[parent][0]) # aka num of parents of this parent
        numstates_parents += find_num_states(numprobs, in_edges)
    return numstates_parents
    
    
def num_qbits_needed_general(graph): #THIS FUNCTION DOES NOT WORK FOR 3 NODE CASE BUT I THINK IT DOES MATCH EQN 22.....
    sumterm = 0
    ancillaterms = []
    varnum = 0
    for var in graph:
        probs = graph[var]
        numprobs = len(probs[1]) # number of states if parent node, number of conditional states if child node
        in_edges = len(graph[var][0]) # aka num of parents
        numstates = find_num_states(numprobs, in_edges)
        sumterm += math.ceil(np.log2(numstates))
        if in_edges > 0:
            parentvarnames = graph[var][0] # list of parent names
            numstates_parents = find_numstates_parents(graph, parentvarnames)
            ancillaterm = numstates_parents/2 + math.ceil(np.log2(numstates)) - 1 #MAYBE THERE SHOULD NOT BE THE LOG2(N) HERE!?!? 
            ancillaterms.append(ancillaterm)
    qbits = sumterm + max(ancillaterms) # equation (22)
    cbits = len(graph) # number of measurements
    return (qbits, cbits)

def num_qbits_needed(graph):
    max_edges = -1 
    for state in graph:
        in_edges = len(graph[state][0]) # aka num of parents
        if in_edges > max_edges:
            max_edges = in_edges
    
    qbits = len(graph) + max_edges - 1 # equation (15)
    cbits = len(graph) # number of measurements
    return (qbits, cbits)

In [22]:
def run_circuit(circuit, output_file='results', draw_circuit=True, use_sim=True, use_noise=False, use_qcomp=False, shots=1024):
    if draw_circuit:
        #print(circuit.draw())
        %config InlineBackend.figure_format
        circuit.draw(output='mpl')

    if use_noise or use_qcomp:
        IBMQ.load_account()
        provider = IBMQ.get_provider('ibm-q')
        qcomp = provider.get_backend('ibmq_16_melbourne')

    if use_sim:
        simulator = Aer.get_backend('qasm_simulator')
        if use_noise:
            noise_model = NoiseModel.from_backend(qcomp)
            basis_gates = noise_model.basis_gates
            coupling_map = qcomp.configuration().coupling_map
            job = execute(circuit, backend=simulator, coupling_map=coupling_map, noise_model=noise_model, basis_gates=basis_gates, shots=shots)
        else:
            job = execute(circuit, backend=simulator, shots=shots)
        print(job.result().get_counts())
        plot_histogram(job.result().get_counts()).savefig(output_file+"-sim.png")


    if use_qcomp:
        job = execute(circuit, backend=qcomp, shots=shots)
        job_monitor(job)
        if(job.status() == JobStatus.ERROR):
            print("Error: Check with IBMQ")
        print(job.result().get_counts())
        plot_histogram(job.result().get_counts()).savefig(output_file+'-qcomp.png')
    return job.result()

In [23]:
def add_cny(circuit, n, angle, control_bits, target, ancilla_bits):
    '''
    Creates generalized cny gate for any n.
    
    circuit:       existing circuit to modify
    n:             number of control bits required (number of "C"s wanted for this gate)
    angle:         angle to rotate by (rad)
    control_bits:  list (len n)
    target:        qbit you want to rotate
    ancilla_bits:  list (len n-1); these are left clean (in same state as initially)
    
    Note: could also try rewriting using Qiskit's built-in mcx gates
    https://qiskit.org/documentation/stubs/qiskit.circuit.QuantumCircuit.mcx.html
    https://qiskit.org/documentation/tutorials/circuits_advanced/1_advanced_circuits.html
    '''
    # assert some things are true
    assert n >= 2
    assert len(control_bits) == n
    assert len(ancilla_bits) == n-1
    
    circuit.barrier() # for visualization
    
    # hardcode first ccx gate
    circuit.ccx(control_bits[0], control_bits[1], ancilla_bits[0])
    
    # loop to create ccx gates
    for i in range(2, n):
        circuit.ccx(control_bits[i], ancilla_bits[i-2], ancilla_bits[i-1])
    
    # add rotate block
    circuit.cry(angle, ancilla_bits[n-2], target)
    
    # loop to create remaining ccx gates
    for i in range(n-1, 1, -1):
        circuit.ccx(control_bits[i], ancilla_bits[i-2], ancilla_bits[i-1])
        
    # hardcode last ccx gate
    circuit.ccx(control_bits[0], control_bits[1], ancilla_bits[0])
    
    circuit.barrier() # for visualization
    
    return circuit


In [46]:
def create_circuit(graph):
    
    # initialize circuit
    (qbits, cbits) = num_qbits_needed(graph)
    qc = QuantumCircuit(qbits, cbits)
    
    bit_assignment = {}
    ancilla_bits = []
    next_free_qbit = 0
    
    # loop and find the parentless nodes (sad orphans), assign their rotations first
    for node in graph:
        number_of_parents = len(graph[node][0])
        
        if number_of_parents == 0:            
            probs = graph[node][1]
            
            qc.ry(angle_from_probability(probs[0], probs[1]), next_free_qbit)
            # keep track of what node is what qbit
            bit_assignment[node] = next_free_qbit 
            next_free_qbit += 1
            
    qc.barrier() # for visualization
    

    # loop and find each node that has all its parents complete and add it
    #TODO a topological sort first would save this unnessecary looping
    #XXX bad code, maybe the worst
    
    while True: # Loop until no states are added
        found_new = False
        
        for node in graph:
            if node not in bit_assignment:
                for parent in graph[node][0]:  # get parents
                    
                    if parent not in bit_assignment:
                        # one of the parents needs to be processed first
                        continue 
                number_of_parents = len(graph[node][0])
                # otherwise, now we found a node we are ready to process
                found_new = True
                
                # TODO handle multi-bit states
                bit_assignment[node] = next_free_qbit # give it a qbit
                target_qbit = next_free_qbit
                next_free_qbit += 1
                
                
                # loop over number of blocks needed for this node
                for i in range(2**(number_of_parents)):
                    # convert i to binary to get current state (eg. |010>, etc.)
                    current_state = bin(i)[2:].zfill(number_of_parents)[::-1]
                    
                    parent_bits = []
                    
                    # loop through node's parents
                    for j, parent in enumerate(graph[node][0]):
                        current_parent_bit = bit_assignment[parent]
                        parent_bits.append(current_parent_bit)
                        
                        if current_state[j] == '0':
                            # we need to add an x gate
                            qc.x(current_parent_bit)
                            
                    while (len(ancilla_bits) < number_of_parents):
                        #XXX we could also just pre-allocate the last 5 bits as ancilla bits..
                        
                        # make sure we have enough bits to assign an ancilla bits
                        ancilla_bits += [next_free_qbit]
                        next_free_qbit += 1
                    
                    # find 2 probs
                    p0 = graph[node][1][i*2]
                    print(graph[node][1], i)
                    p1 = graph[node][1][i*2+1]
                    
                    # calculate angle
                    angle = angle_from_probability(p0, p1)
                    
                    # now, add rotation gate
                    if number_of_parents == 1:
                        print(parent_bits)
                        
                        qc.cry(angle, parent_bits[0], target_qbit)
                        
                    else:
                        qc = add_cny(qc, number_of_parents, angle, parent_bits, target_qbit, ancilla_bits[:(number_of_parents-1)])
                    
                    # add second wall of x gates
                    for j, parent in enumerate(graph[node][0]):
                        
                        if current_state[j] == '0':
                            current_parent_bit = bit_assignment[parent]
                            
                            # we need to add an x gate
                            qc.x(current_parent_bit)
                    qc.barrier()
        # done!!!
        if not found_new:
            # completed a full loop over the nodes and all were added
            break

    c_counter = 0
    
    for bit in bit_assignment:
        qc.measure(bit_assignment[bit], c_counter)
        c_counter += 1
        
    return qc

In [28]:
def generate_test_circuit():
    
    
    
    qc = QuantumCircuit(20, 5)
    
    qc = add_cny(qc, 10, pi, [0,1,2,3,4,5,6,7,8,9], 19, [10,11,12,13,14,15,16,17,18])
    
    
    return qc

In [47]:
circuit = create_circuit(oil_graph)


[0.3, 0.7, 0.8, 0.2] 0
[0]
[0.3, 0.7, 0.8, 0.2] 1
[0]
[0.9, 0.1, 0.5, 0.5, 0.4, 0.6, 0.2, 0.8] 0
[0.9, 0.1, 0.5, 0.5, 0.4, 0.6, 0.2, 0.8] 1
[0.9, 0.1, 0.5, 0.5, 0.4, 0.6, 0.2, 0.8] 2
[0.9, 0.1, 0.5, 0.5, 0.4, 0.6, 0.2, 0.8] 3


In [ ]:
result = run_circuit(circuit, "lesser_model", shots=1024*8, use_sim=True, use_qcomp=False, use_noise=False)


In [48]:
#%config InlineBackend.figure_format
#circuit.draw(output='mpl')
circuit.draw()

┌──────────┐  ░ ┌───┐              ┌───┐ ░                ░       ░      »
q_0: ─┤ RY(pi/3) ├──░─┤ X ├──────■───────┤ X ├─░───────■────────░───────░──────»
     ┌┴──────────┴┐ ░ └───┘      │       └───┘ ░       │        ░ ┌───┐ ░      »
q_1: ┤ RY(1.3694) ├─░────────────┼─────────────░───────┼────────░─┤ X ├─░───■──»
     └────────────┘ ░      ┌─────┴──────┐      ░ ┌─────┴──────┐ ░ ├───┤ ░   │  »
q_2: ───────────────░──────┤ RY(1.9823) ├──────░─┤ RY(0.9273) ├─░─┤ X ├─░───■──»
                    ░      └────────────┘      ░ └────────────┘ ░ └───┘ ░ ┌─┴─┐»
q_3: ───────────────░──────────────────────────░────────────────░───────░─┤ X ├»
                    ░                          ░                ░       ░ └───┘»
q_4: ───────────────░──────────────────────────░────────────────░───────░──────»
                    ░                          ░                ░       ░      »
c_0: ══════════════════════════════════════════════════════════════════════════»
                                                                               »
c_1: ══════════════════════════════════════════════════════════════════════════»
                                                                               »
c_2: ══════════════════════════════════════════════════════════════════════════»
                                                                               »
c_3: ══════════════════════════════════════════════════════════════════════════»
                                                                               »
«                         ░       ░       ░                        ░       ░ »
«q_0: ────────────────────░───────░───────░────────────────────────░───────░─»
«                         ░ ┌───┐ ░       ░                        ░       ░ »
«q_1: ────────────────■───░─┤ X ├─░───────░───■────────────────■───░───────░─»
«                     │   ░ ├───┤ ░ ┌───┐ ░   │                │   ░ ┌───┐ ░ »
«q_2: ────────────────■───░─┤ X ├─░─┤ X ├─░───■────────────────■───░─┤ X ├─░─»
«                   ┌─┴─┐ ░ └───┘ ░ └───┘ ░ ┌─┴─┐            ┌─┴─┐ ░ └───┘ ░ »
«q_3: ──────■───────┤ X ├─░───────░───────░─┤ X ├─────■──────┤ X ├─░───────░─»
«     ┌─────┴──────┐└───┘ ░       ░       ░ └───┘┌────┴─────┐└───┘ ░       ░ »
«q_4: ┤ RY(0.6435) ├──────░───────░───────░──────┤ RY(pi/2) ├──────░───────░─»
«     └────────────┘      ░       ░       ░      └──────────┘      ░       ░ »
«c_0: ═══════════════════════════════════════════════════════════════════════»
«                                                                            »
«c_1: ═══════════════════════════════════════════════════════════════════════»
«                                                                            »
«c_2: ═══════════════════════════════════════════════════════════════════════»
«                                                                            »
«c_3: ═══════════════════════════════════════════════════════════════════════»
«                                                                            »
«           ░                          ░       ░  ░                          ░ »
«q_0: ──────░──────────────────────────░───────░──░──────────────────────────░─»
«     ┌───┐ ░                          ░ ┌───┐ ░  ░                          ░ »
«q_1: ┤ X ├─░───■──────────────────■───░─┤ X ├─░──░───■──────────────────■───░─»
«     └───┘ ░   │                  │   ░ └───┘ ░  ░   │                  │   ░ »
«q_2: ──────░───■──────────────────■───░───────░──░───■──────────────────■───░─»
«           ░ ┌─┴─┐              ┌─┴─┐ ░       ░  ░ ┌─┴─┐              ┌─┴─┐ ░ »
«q_3: ──────░─┤ X ├──────■───────┤ X ├─░───────░──░─┤ X ├──────■───────┤ X ├─░─»
«           ░ └───┘┌─────┴──────┐└───┘ ░       ░  ░ └───┘┌─────┴──────┐└───┘ ░ »
«q_4: ──────░──────┤ RY(1.7722) ├──────░───────░──░──────┤ RY(2.2143) ├──────░─»
«           ░      └────────────┘      ░       ░  ░      └────────────┘      ░ »
«c_0: ═════════════════════════════════════════════════════════════════════

In [ ]:
plot_histogram(result.get_counts())

In [ ]:
generate_test_circuit().draw()

In [ ]:
oil_graph = {'IR': ([], [.75, .25]), # From 2004.14803, Fig 10
         'SM': (['IR'], [.3, .7, .8, .2]), #P(0|!A), P(1|!A), P(0|A), P(1|A)
         'OI': ([], [.6, .4]),
         'SP': (['OI', 'SM'], [.9, .1, .5, .5, .4, .6, .2, .8])
        }

for i, nodes in enumerate(oil_graph):
    print(i, nodes)
